In [6]:
# ===================================================
# Graduate Programme – Panel Assessment
# Fund Controlling (Complete Python Solution)
# ===================================================

import pandas as pd
import numpy as np
import plotly.express as px
from ipywidgets import interact, FloatSlider
from IPython.display import display

# ---- Load dataset ----
file_path = "/2025 Graduate Programme - Assessment Data Set- Fund Controlling.xlsx"
data = pd.read_excel(file_path, sheet_name=" Dataset")

print("✅ Dataset Loaded Successfully")
print("Columns:", data.columns.tolist())
print(data.head())

# ===================================================
# 1. Dashboard of Fund Performance
# ===================================================
fig = px.line(
    data,
    x="Month", y="Monthly Return",
    color="Fund Name",
    markers=True,
    title="Fund Performance Over Time"
)
fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Monthly Return",
    legend_title="Fund Name"
)
fig.show()

# ===================================================
# 2. Risk vs Return Comparison
# ===================================================
risk_return = data.groupby("Fund Name").agg({
    "Monthly Return": "mean",
    "Volatility": "mean"
}).reset_index()

# Bubble sizes proportional to absolute return
risk_return["Size"] = (risk_return["Monthly Return"].abs() * 100).clip(lower=5)

fig2 = px.scatter(
    risk_return,
    x="Volatility",
    y="Monthly Return",
    color="Fund Name",
    size="Size",
    hover_name="Fund Name",
    text="Fund Name",
    title="Risk vs Return Comparison"
)
fig2.update_traces(textposition="top center")
fig2.show()

# ===================================================
# 3. Average Monthly Returns
# ===================================================
avg_returns = data.groupby("Fund Name")["Monthly Return"].mean().reset_index()
avg_returns.columns = ["Fund Name", "Avg Monthly Return"]

print("\n📊 Average Monthly Returns:")
display(avg_returns)

# ===================================================
# 4. Highest/Lowest Risk Investments
# ===================================================
highest_risk = risk_return.loc[risk_return["Volatility"].idxmax()]
lowest_risk = risk_return.loc[risk_return["Volatility"].idxmin()]

print("\n⚠️ Highest Risk Investment:")
display(highest_risk.to_frame().T)

print("\n✅ Lowest Risk Investment:")
display(lowest_risk.to_frame().T)

# ===================================================
# 5. Monthly Return Ranking (Top 3 / Bottom 3)
# ===================================================
ranking_counts = []
for month, subdf in data.groupby("Month"):
    subdf = subdf.sort_values("Monthly Return", ascending=False)
    subdf["Rank"] = range(1, len(subdf)+1)
    ranking_counts.append(subdf)

ranking_df = pd.concat(ranking_counts)

top3_counts = ranking_df[ranking_df["Rank"] <= 3].groupby("Fund Name").size()
bottom3_counts = ranking_df[ranking_df["Rank"] > (len(risk_return) - 3)].groupby("Fund Name").size()

print("\n🏆 Top 3 appearances:")
display(top3_counts)

print("\n⬇️ Bottom 3 appearances:")
display(bottom3_counts)

# ===================================================
# 6. Asset Allocation Consistency
# ===================================================
alloc_cols = ["Equity Allocation (%)", "Bond Allocation (%)", "Cash Allocation (%)"]
alloc_stability = data.groupby("Fund Name")[alloc_cols].std().mean(axis=1).reset_index()
alloc_stability.columns = ["Fund Name","Avg Allocation Std"]

most_stable = alloc_stability.loc[alloc_stability["Avg Allocation Std"].idxmin()]
least_stable = alloc_stability.loc[alloc_stability["Avg Allocation Std"].idxmax()]

print("\n📉 Most Stable Allocation:")
display(most_stable.to_frame().T)

print("\n📈 Least Stable Allocation:")
display(least_stable.to_frame().T)

# ===================================================
# 7. Cash Allocation Validation
# ===================================================
data["Total Alloc"] = data[alloc_cols].sum(axis=1)
data["Cash Corrected"] = data["Cash Allocation (%)"]

# Fix negatives
data.loc[data["Cash Corrected"] < 0, "Cash Corrected"] = 0
# Adjust so total = 100
data["Cash Corrected"] = data["Cash Corrected"] + (100 - data["Total Alloc"])

print("\n💵 Cash Allocation Validation (sample rows):")
display(data[[
    "Month","Fund Name",
    "Equity Allocation (%)","Bond Allocation (%)",
    "Cash Allocation (%)","Cash Corrected","Total Alloc"
]].head())

# ===================================================
# 8. Fund Summary Report
# ===================================================
summary = data.groupby("Fund Name").agg({
    "Monthly Return": "mean",
    "Volatility": "mean",
    "Equity Allocation (%)": "mean",
    "Bond Allocation (%)": "mean",
    "Cash Corrected": "mean"
}).reset_index()

summary.columns = ["Fund Name","Avg Return","Avg Risk","Avg Equity","Avg Bonds","Avg Cash"]

print("\n📑 Fund Summary Report:")
display(summary)

# ===================================================
# Part 2: Interactive Tool for Investor Selection
# ===================================================
def fund_selector(max_risk, min_return):
    """Filter funds by investor’s risk appetite and return expectation"""
    filtered = summary[(summary["Avg Risk"] <= max_risk) &
                       (summary["Avg Return"] >= min_return)]
    if filtered.empty:
        print("⚠️ No funds match this criteria.")
    else:
        print(f"\n🎯 Funds with Risk ≤ {max_risk:.3f} and Return ≥ {min_return:.3f}:")
        display(filtered)

# Interactive sliders
interact(
    fund_selector,
    max_risk=FloatSlider(value=0.05, min=0, max=0.1, step=0.005, description="Max Risk"),
    min_return=FloatSlider(value=0.0, min=-0.05, max=0.05, step=0.005, description="Min Return")
)


✅ Dataset Loaded Successfully
Columns: ['Fund Name', 'Month', 'Monthly Return', 'Volatility', 'Equity Allocation (%)', 'Bond Allocation (%)', 'Cash Allocation (%)']
  Fund Name     Month  Monthly Return  Volatility  Equity Allocation (%)  \
0     Alpha  Jan-2023          0.0199      0.0320                  63.95   
1      Beta  Jan-2023          0.0248      0.0127                  47.84   
2     Delta  Jan-2023          0.0062      0.0143                  59.58   
3   Epsilon  Jan-2023         -0.0036      0.0148                  75.94   
4       Eta  Jan-2023         -0.0041      0.0462                  65.50   

   Bond Allocation (%)  Cash Allocation (%)  
0                16.24                19.81  
1                11.81                40.35  
2                49.43                -9.00  
3                34.26               -10.20  
4                39.04                -4.54  



📊 Average Monthly Returns:


,Fund Name,Avg Monthly Return
0,Alpha,0.011633
1,Beta,0.016650
2,Delta,0.007458
3,Epsilon,0.016325
4,Eta,-0.000633
5,Gamma,0.004517
6,Zeta,0.000050



⚠️ Highest Risk Investment:


,Fund Name,Monthly Return,Volatility,Size
4,Eta,-0.000633,0.047942,5.0


...

,Month,Fund Name,Equity Allocation (%),Bond Allocation (%),Cash Allocation (%),Cash Corrected,Total Alloc
0,Jan-2023,Alpha,63.95,16.24,19.81,1.981000e+01,100.00
1,Jan-2023,Beta,47.84,11.81,40.35,4.035000e+01,100.00
2,Jan-2023,Delta,59.58,49.43,-9.00,-1.000000e-02,100.01
3,Jan-2023,Epsilon,75.94,34.26,-10.20,1.421085e-14,100.00
4,Jan-2023,Eta,65.50,39.04,-4.54,1.421085e-14,100.00



📑 Fund Summary Report:


,Fund Name,Avg Return,Avg Risk,Avg Equity,Avg Bonds,Avg Cash
0,Alpha,0.011633,0.019050,58.260000,30.441667,14.788333
1,Beta,0.016650,0.023483,58.050000,24.080833,19.016667
2,Delta,0.007458,0.021175,58.665000,37.010833,7.435833
3,Epsilon,0.016325,0.022300,61.493333,34.115000,10.220833
4,Eta,-0.000633,0.047942,53.535000,27.425833,20.723333
5,Gamma,0.004517,0.027225,58.660833,24.370833,17.606667
6,Zeta,0.000050,0.046917,64.295833,31.400833,9.053333


interactive(children=(FloatSlider(value=0.05, description='Max Risk', max=0.1, step=0.005), FloatSlider(value=…

<function __main__.fund_selector(max_risk, min_return)>